In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import os
import time
import json
from tqdm import tqdm
from lxml.html import fromstring
from itertools import cycle
import traceback

In [2]:
def get_brands():
    page = requests.get("https://www.gsmarena.com/makers.php3")
    soup = BeautifulSoup(page.text, 'html.parser')
    table = soup.find_all('table')[0]
    table_a = table.find_all('a')
    phones_brands=[]
    for a in table_a:
        temp = [a['href'].split('-')[0], a.find('span').text.split(' ')[0], a['href']]
        phones_brands.append(temp)
    return phones_brands

In [3]:
def get_model_links(sub_link):
    links = []
    nav_link = []
    main_link= 'https://www.gsmarena.com/'
    phone_brand_link = main_link+sub_link
    page = requests.get(phone_brand_link)
    soup = BeautifulSoup(page.text, 'html.parser')

    nav_data = soup.find(class_='nav-pages')
    if not nav_data:
        nav_link.append(phone_brand_link)
    else:
        nav_link = nav_data.findAll('a')
        nav_link = [link['href'] for link in nav_link]
        #nav_link.append(phone_brand_link)
        #nav_link.insert(0, nav_link.pop())
    for link in nav_link:
        page = requests.get(main_link+link)
        soup = BeautifulSoup(page.text, 'html.parser')
        data = soup.find(class_='section-body')
        for line1 in data.findAll('a'):
            links.append(line1['href'])
    return links


In [4]:
def get_models_specs(sub_link,proxy):
    phone_data = {}
    link='https://www.gsmarena.com/'+sub_link
    #soup = self.crawl_html_page(link)
    #proxies={"http": "http://103.135.251.57:8080",
    #       "https": "http://103.135.251.57:8080"}
    page = requests.get(link,proxies=proxy)
    soup = BeautifulSoup(page.text, 'html.parser')
    phone_brand = 'acer'

    model_name = soup.find(class_='specs-phone-name-title').text
    model_img_html = soup.find(class_='specs-photo-main')
    model_img = model_img_html.find('img')['src']
    phone_data.update({"Brand": phone_brand})
    phone_data.update({"Model Name": model_name})
    phone_data.update({"Model Image": model_img})
    temp = []
    for data1 in range(len(soup.findAll('table'))):
        table = soup.findAll('table')[data1]
        for line in table.findAll('tr'):
            temp = []
            for l in line.findAll('td'):
                text = l.getText()
                text = text.strip()
                text = text.lstrip()
                text = text.rstrip()
                text = text.replace("\n", "")
                temp.append(text)
                if temp[0] in phone_data.keys():
                    temp[0] = temp[0] + '_1'
                #if temp[0] not in self.features:
                #    self.features.append(temp[0])
            if not temp:
                continue
            else:
                phone_data.update({temp[0]: temp[1]})
    return phone_data

In [5]:
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:100]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            try:
                requests.get("https://www.google.com",proxies={'http': 'http://'+proxy,'https': 'http://'+proxy})
                proxies.add(proxy)   
            except:
                pass
    return proxies

In [6]:
phone_brands = get_brands()

In [7]:
phone_brands[:5]

[['acer', '100', 'acer-phones-59.php'],
 ['alcatel', '399', 'alcatel-phones-5.php'],
 ['allview', '157', 'allview-phones-88.php'],
 ['amazon', '19', 'amazon-phones-76.php'],
 ['amoi', '47', 'amoi-phones-28.php']]

In [ ]:
proxies = get_proxies()
proxy_pool = cycle(proxies)

In [ ]:
len(proxies)

In [ ]:
for entry in phone_brands[:1]: #for entry in phone_brands:
    brand_name = entry[0]
    brand_link = entry[2]
    specs_dict={}
    model_specs=[]
    count=0
    proxy = next(proxy_pool)
    prox={'http': 'http://'+proxy,'https': 'http://'+proxy}
    prox.pop("socks",None)
    for i in get_model_links(brand_link):
        count=count+1
        print(count)
        if count%5==0:
            proxy = next(proxy_pool)
            print(proxy)
            prox={'http': 'http://'+proxy,'https': 'http://'+proxy}
            prox.pop("socks",None)
        model_specs.append(get_models_specs(i,prox))
    specs_dict[brand_name]=model_specs

In [ ]:
len(model_specs)